# Analysis of `vehicle-zone-tracking.xml`

In [ ]:
import pprint, math, datetime
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

# scenario = "berlin-1pct"
scenario = "charlottenburg-no-meso-periodic"
# scenario = "normal-output"

def get_total_trips():
    statistics_file = f"{scenario}/statistics.xml"
    for event, element in etree.iterparse(statistics_file, tag="vehicles"):
        return int(element.attrib["inserted"])

def get_total_persons():
    trips_info_file = f"{scenario}/trips-info.xml"
    persons = []
    for event, element in etree.iterparse(trips_info_file, tag="tripinfo"):
        person, counter = element.attrib["id"].split("_")
        if person not in persons:
            persons.append(person)
        
    return len(persons)

total_trips = get_total_trips()
print("Total trips:", total_trips)

total_persons = get_total_persons()
print("Total persons:", total_persons)

In [ ]:
def get_distances_per_zone(file_path, freeze_zones=False):
    distances_per_zone = {}
  
    for event, element in etree.iterparse(file_path, tag="timestep"):
        current_zone_timestep = element.attrib["zone-timestep"]

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            speed = float(vehicle.attrib["speed"])
            v_timestep = vehicle.attrib["zone-timestep"]

            distances = {}
            if vid in distances_per_zone:
                distances = distances_per_zone[vid]

            for polygon in vehicle.getchildren():
                pid_parts = polygon.attrib["id"].split("_")
                pid = pid_parts[0]
                p_timestep = pid_parts[1]

                if freeze_zones:
                    if p_timestep != v_timestep:
                        continue
                else:
                    if p_timestep != current_zone_timestep:
                        continue

                if pid.startswith("hole"):
                    hole, hole_counter, zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] -= speed
                    else:
                        distances[zone] = -speed
                else:
                    zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] += speed
                    else:
                        distances[zone] = speed
            
            distances_per_zone[vid] = distances

        element.clear()

    return distances_per_zone

distances = get_distances_per_zone(f"{scenario}/vehicle-zone-tracking.xml")

print("Done")

### Distances per trip

In [ ]:
# pprint.pprint(distances)

### Distances per person

In [ ]:
distances_pp = {}

for trip in distances:
    person, counter = trip.split("_")

    if person in distances_pp:
        persodistances = distances_pp[person]
        trip_distances = distances[trip]

        for zone in trip_distances:
            if zone in persodistances:
                persodistances[zone] += trip_distances[zone]
            else:
                persodistances[zone] = trip_distances[zone]
                
        distances_pp[person] = persodistances

    else:
        distances_pp[person] = distances[trip]

# pprint.pprint(distances_pp)

### Total trips per zone

In [ ]:
total_trips_pz = {}

for trip in distances:
    distances_pz = distances[trip]
    for zone in distances_pz:
        if zone in total_trips_pz:
            total_trips_pz[zone] += 1
        else:
            total_trips_pz[zone] = 1

for zone in total_trips_pz:
    print(f"Zone {zone}:", total_trips_pz[zone])

# print(total_trips_pz)

### Total persons per zone

In [ ]:
total_persons_pz = {}

for trip in distances:
    person, counter = trip.split("_")
    distances_pz = distances[trip]
    for zone in distances_pz:
        if zone in total_persons_pz:
            if person not in total_persons_pz[zone]:
                total_persons_pz[zone].append(person)
        else:
            total_persons_pz[zone] = [person]

for zone in total_persons_pz:
    total_persons_pz[zone] = len(total_persons_pz[zone])
    print(f"Zone {zone}:", total_persons_pz[zone])
    
# print(total_persons_pz)

### Total distances per zone

In [ ]:
total_distances_pz = {}

for trip in distances:
    distances_pz = distances[trip]
    for zone in distances_pz:
        if zone in total_distances_pz:
            total_distances_pz[zone]["distance"] += distances_pz[zone] / 1000
            total_distances_pz[zone]["n"] += 1
        else:
            total_distances_pz[zone] = {
                "distance": distances_pz[zone] / 1000,
                "n": 1
            } 

for zone in total_distances_pz:
    total_distances_pz[zone] = {
        "total": total_distances_pz[zone]["distance"],
        "n": total_distances_pz[zone]["n"]
    }
    print(f"Zone {zone}:", total_distances_pz[zone]["total"], "km")

# pprint.pprint(total_distances_pz)

### Average distance per zone per trip

In [ ]:
avg_distances_pz = {}

for trip in distances:
    distances_pz = distances[trip]
    for zone in distances_pz:
        if zone in avg_distances_pz:
            avg_distances_pz[zone]["distance"] += distances_pz[zone]
            avg_distances_pz[zone]["n"] += 1
        else:
            avg_distances_pz[zone] = {
                "distance": distances_pz[zone],
                "n": 1
            }
    
for zone in avg_distances_pz:
    avg_distances_pz[zone] = {
        "avg": avg_distances_pz[zone]["distance"] / avg_distances_pz[zone]["n"],
        "n": avg_distances_pz[zone]["n"]
    }
    print(f"Zone {zone}:", avg_distances_pz[zone]["avg"], "m")
# pprint.pprint(avg_distances_pz)

### Average distance per zone per person

In [ ]:
avg_distances_pz = {}

for person in distances_pp:
    distances_pz = distances_pp[person]
    for zone in distances_pz:
        if zone in avg_distances_pz:
            avg_distances_pz[zone]["distance"] += distances_pz[zone]
            avg_distances_pz[zone]["n"] += 1
        else:
            avg_distances_pz[zone] = {
                "distance": distances_pz[zone],
                "n": 1
            }

for zone in avg_distances_pz:
    avg_distances_pz[zone] = {
        "avg": avg_distances_pz[zone]["distance"] / avg_distances_pz[zone]["n"],
        "n": avg_distances_pz[zone]["n"]
    }
    
    print(f"Zone {zone}:", avg_distances_pz[zone]["avg"], "m")
    
# pprint.pprint(avg_distances_pz)